In [2]:
pip install -U classiq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing insta

In [3]:
import classiq
classiq.authenticate()

Your user code: CGKF-LGQL
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=CGKF-LGQL


## Implementing a Toy Problem : Quantum Walk (Linear Graph)

This can be thought of n coupled oscillators connected in a linear fashion. We are just traversing the oscillators in this problem. Later we will use the algoithm we developed from the paper in the actual problem

In [59]:
from classiq import *

size = 4

@qfunc
def prepare_minus(x: QBit):
  X(x)
  H(x)


@qfunc
def diffuzer_oracle(aux: Output[QNum],x:QNum):
  aux^=(x!=0)


@qfunc
def zero_diffuzer(x: QNum):
  aux = QNum('aux')
  allocate(1,aux)
  within_apply(compute=lambda: prepare_minus(aux),
              action=lambda: diffuzer_oracle)



def W_iteration(i:int,vertices: QNum, adjacent_vertices:QNum):
    prob = [0] * 16
    if i == 0:
        prob[1] = 1.0
    elif i == 15:
        prob[14] = 1.0
    else:
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5
    print(f'State={i}, prob vec ={prob}')

    control(ctrl=vertices==i,
            operand=lambda: within_apply(
              compute= lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
              action= lambda: zero_diffuzer(adjacent_vertices)))


@qfunc
def W_operator(vertices:QNum, adjacent_vertices: QNum):
    for i in range(2**size):
      W_iteration(i,vertices,adjacent_vertices)


## Time Evolution Part

@qfunc
def edge_oracle(res:Output[QBit], vertices: QNum, adjacent_vertices: QNum):
  res |= (((vertices+adjacent_vertices)%2) ==1)


@qfunc
def bitwise_swap(x: QArray[QBit], y:QArray[QBit]):
  repeat(count= x.len,
    iteration= lambda i: SWAP(x[i],y[i]))


@qfunc
def S_operator(vertices:QNum, adjacent_vertices: QNum):
    res = QNum('res')
    edge_oracle(res,vertices,adjacent_vertices)
    control(ctrl= res==1,
        operand= lambda: bitwise_swap(vertices,adjacent_vertices))


## Main Function

@qfunc
def main(vertices:Output[QNum], adjacent_vertices:Output[QNum]):

  allocate(size,vertices)
  hadamard_transform(vertices)
  allocate(size,adjacent_vertices)

  W_operator(vertices,adjacent_vertices)
  S_operator(vertices,adjacent_vertices)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

State=0, prob vec =[0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=1, prob vec =[0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=2, prob vec =[0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=3, prob vec =[0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=4, prob vec =[0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=5, prob vec =[0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0]
State=6, prob vec =[0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0, 0]
State=7, prob vec =[0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0, 0]
State=8, prob vec =[0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0, 0]
State=9, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0, 0]
State=10, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0, 0]
State=11, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0, 0]
State=12, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0, 0.5, 0, 0]
State=13, prob vec =[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.5, 0

<ipython-input-59-d0db1198a51d>:36: DeprecationWarning: Parameter 'operand' of function 'control' has been renamed to 'stmt_block'. Parameter 'operand' will be deprecated in a future release.
Hint: Change `control(ctrl=..., operand=...)` to `control(ctrl=..., stmt_block=...)` or `control(..., ...)`.
  control(ctrl=vertices==i,
<ipython-input-59-d0db1198a51d>:37: DeprecationWarning: Parameters 'compute' and 'action' of function 'within_apply' have been renamed to 'within' and 'apply' respectively. Parameters 'compute' and 'action' will be deprecated in a future release.
Hint: Change `within_apply(compute=..., action=...)` to `within_apply(within=..., apply=...)` or `within_apply(..., ...)`.
  operand=lambda: within_apply(
<ipython-input-59-d0db1198a51d>:20: DeprecationWarning: Parameters 'compute' and 'action' of function 'within_apply' have been renamed to 'within' and 'apply' respectively. Parameters 'compute' and 'action' will be deprecated in a future release.
Hint: Change `within_a

Opening: https://platform.classiq.io/circuit/0581776a-02d5-4068-ae1b-188ed8061c16?version=0.44.0


## Implementation of the Main Problem

In [79]:
def sqrt_mb_gate_internal(j: QBit, k: QBit, kappas: CArray):
    """Implements the internal sqrt(MB) gate based on the given problem."""

    # Iterate over all possible vertex states
    for vertex_index in range(2**size):
        amplitude_jj = sqrt(kappas[vertex_index][vertex_index])
        amplitude_jk = sqrt(kappas[vertex_index][k])

        control(ctrl=(j == vertex_index), operand=lambda: apply_amplitude_gate(amplitude_jj, j))

        if vertex_index < 2**size - 1:  # Ensuring we're within bounds
            control(ctrl=(k == vertex_index), operand=lambda: apply_amplitude_gate(amplitude_jk, k))
            control(ctrl=(k == vertex_index), operand=lambda: apply_phase_gate(-amplitude_jk, k))

# You can now use sqrt_mb_gate_internal in your code where you would have used sqrt_mb_gate before.

In [80]:
from classiq import *

In [81]:
import numpy as np
from classiq import *

# Set the size (number of qubits)
size = 4

# Generate a random symmetric matrix K with non-negative values
def generate_random_symmetric_matrix(size):
    mat = np.random.rand(size, size)
    return (mat + mat.T) / 2

K = generate_random_symmetric_matrix(2**size)

@qfunc
def sqrt_mb_gate(j: CInt, k: CInt, K: CArray):
    """Implements the sqrt(MB) gate based on the given problem."""
    for i in range(2**size):
        if i == j:
            # Case where j = k
            amplitude = np.sqrt(K[i][i])
            apply_amplitude_gate(amplitude, k)
        else:
            # Case where j < k
            amplitude = np.sqrt(K[i][j])
            apply_amplitude_gate(amplitude, k)
            # Subtract from |k⟩ state
            control(ctrl=k == j,
                    operand=lambda: apply_phase_gate(-amplitude, k))

@qfunc
def W_iteration(i: CInt, vertices: QNum, adjacent_vertices: QNum):
    """Modified iteration function with sqrt(MB) applied."""
    prob = [0] * 16
    if i == 0:
        prob[1] = 1.0
    elif i == 15:
        prob[14] = 1.0
    else:
        prob[i - 1] = 0.5
        prob[i + 1] = 0.5

    # Ensure the correct application of control and gates
    control(ctrl=(vertices == i),
            operand=lambda: within_apply(
                compute=lambda: inplace_prepare_state(probabilities=prob, bound=0.01, target=adjacent_vertices),
                action=lambda: sqrt_mb_gate(i, adjacent_vertices, K)))

@qfunc
def W_operator(vertices: QNum, adjacent_vertices: QNum):
    """Applies the W operator for the quantum walk."""
    for i in range(2**size):
        W_iteration(CInt(i), vertices, adjacent_vertices)


In [82]:
import numpy as np

# Assume M is your mass or kinetic matrix
M = np.array([[1, 0, 0],  # Fill in with your matrix data
              [0, 1, 0],
              [0, 0, 1]])

# Step 1: Diagonalize M
eigenvalues, eigenvectors = np.linalg.eigh(M)

# Step 2: Compute the square root of the diagonal matrix of eigenvalues
sqrt_eigenvalues = np.sqrt(eigenvalues)
sqrt_Lambda = np.diag(sqrt_eigenvalues)

# Step 3: Reconstruct sqrt(M)
sqrt_M = np.dot(eigenvectors, np.dot(sqrt_Lambda, eigenvectors.T))

# print("The sqrt(M) matrix is:\n", sqrt_M)


from classiq import *

# Let's assume sqrt_M is the precomputed sqrt(M) matrix

@qfunc
def prepare_velocity_state(vertices: QNum, sqrt_M):
    for i in range(vertices.size):
        amplitude = sqrt_M[i]  # Extract amplitude corresponding to velocity
        apply_amplitude_gate(amplitude, vertices[i])  # Apply gate based on sqrt(M)

# This is an abstract implementation; the actual gate sequences will depend on your exact needs and the structure of sqrt(M).



In [83]:
import numpy as np
from classiq import *

size = 4

# Example data (replace with actual problem data)
M = np.array([[2, 1, 0, 0],
              [1, 2, 1, 0],
              [0, 1, 2, 1],
              [0, 0, 1, 2]])

H = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

# Compute sqrt_M
eigenvalues, eigenvectors = np.linalg.eigh(M)
sqrt_eigenvalues = np.sqrt(eigenvalues)
sqrt_Lambda = np.diag(sqrt_eigenvalues)
sqrt_M = np.dot(eigenvectors, np.dot(sqrt_Lambda, eigenvectors.T))

# Compute B_dagger
B_dagger = np.conjugate(H.T)

# Functions to use precomputed sqrt_M and B_dagger
@qfunc
def prepare_velocity_state(vertices: QNum, sqrt_M: CArray):
    for i in range(vertices.size):
        amplitude = sqrt_M[i]  # Use sqrt(M) for initial velocity state
        apply_amplitude_gate(amplitude, vertices[i])

@qfunc
def prepare_displacement_state(vertices: QNum, B_dagger: CArray):
    for i in range(vertices.size):
        amplitude = B_dagger[i]  # Use B^\dagger for displacement state
        apply_amplitude_gate(amplitude, vertices[i])

# Update to use the computed matrices
@qfunc
def prepare_initial_state(vertices: QNum):
    aux = QNum('aux')
    allocate(1, aux)

    # Velocity part of |psi(0)>
    within_apply(compute=lambda: prepare_velocity_state(vertices, sqrt_M),
                 action=lambda: sqrt_mb_gate(vertices, aux, K))

    # Displacement part of |psi(0)>
    within_apply(compute=lambda: prepare_displacement_state(vertices, B_dagger),
                 action=lambda: apply_phase_gate(1j, vertices))  # Introduce the i factor

    # Combine both parts and normalize
    combine_and_normalize(vertices, aux)

# Ensure the combine_and_normalize function is compatible with the quantum framework and problem constraints


In [84]:
import numpy as np
from classiq import QNum, RZ, RY

def apply_unitary_gate(amplitude: CReal, vertices: QNum):
    """Applies a unitary gate with the given amplitude to the vertices qubit register."""
    angle = amplitude.angle()  # Phase component
    magnitude = amplitude.magnitude()  # Magnitude component

    RZ(vertices, angle)  # Apply phase shift
    RY(vertices, 2 * np.arcsin(magnitude))  # Apply rotation for magnitude

def apply_hamiltonian_step(vertices: QNum, h_value: CReal, t: CReal):
    """Apply a single step of the Hamiltonian evolution."""
    cosine_term = cos(t)  # Classiq's cos function
    sine_term = -1j * sin(t) * h_value  # Classiq's sin function

    apply_unitary_gate(cosine_term, vertices)
    apply_unitary_gate(sine_term, vertices)

@qfunc
def apply_time_evolution(vertices: QNum, time: CReal):
    """Apply the time evolution operator exp(-iHt) to the quantum state."""
    for j in range(2**size):
        h_value = H_value(j, vertices)
        apply_hamiltonian_step(vertices, h_value, time)


In [85]:
!pip install scipy
import numpy as np
from scipy.linalg import sqrtm

In [86]:
@qfunc
def apply_time_evolution(vertices: QNum, time: CReal):
    # Applying cosine term
    cos_term = cos(time)
    for i in range(2**size):
        control(ctrl=vertices==i, operand=lambda: apply_amplitude_gate(cos_term, vertices))

    # Applying sine term with Hamiltonian H
    sin_term = sin(time)
    for j in range(2**size):
        control(ctrl=vertices==j, operand=lambda: hamiltonian_gate(sin_term, vertices, j))

@qfunc
def hamiltonian_gate(sin_term: CReal, vertices: QNum, j: CInt):
    """Applies the Hamiltonian operator H scaled by sin(t) to the state."""
    # Assuming the Hamiltonian H is sparse and represented in some form
    amplitude = -i * sin_term * H_value(j, vertices)  # H_value(j, vertices) should be defined
    apply_phase_gate(amplitude, vertices)

M = np.array([[1.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])  # Diagonal mass matrix
F = np.array([[0.0, 1.0, 0.0], [1.0, 0.0, 1.0], [0.0, 1.0, 0.0]])  # Coupling matrix

# Compute the inverse and square root of M
M_inv = np.linalg.inv(M)
sqrt_M_inv = np.sqrt(M_inv)

# Compute A
A = sqrt_M_inv @ F @ sqrt_M_inv

# Compute sqrt(A) using scipy
sqrt_A = sqrtm(A)

def H_value(j: CInt, vertices: QNum):
    """Returns the Hamiltonian value for the given vertex index j."""
    # Extract the Hamiltonian matrix value H[j, j] based on sqrt(A)
    # Assuming vertices.index provides the corresponding index for H
    h_value = sqrt_A[j, j]

    # Since this is a placeholder, you should replace this with actual computation
    return CReal(-h_value)  # Return the negative value of sqrt(A) for H = -sqrt(A)

@qfunc
def evolve_system(vertices: QNum, time: CReal):
    # Apply the time evolution operator
    apply_time_evolution(vertices, time)

import numpy as np

# Precompute values
time_value = 5.0
cos_value = np.cos(time_value)
sin_value = np.sin(time_value)

In [87]:
@qfunc
def main(vertices: Output[QNum], adjacent_vertices: Output[QNum]):
    allocate(size, vertices)
    hadamard_transform(vertices)
    allocate(size, adjacent_vertices)

    time = 5.0

    # Apply the W operator for the quantum walk
    W_operator(vertices, adjacent_vertices)

    # Evolve the system
    evolve_system(vertices, time)

# Create and synthesize the model
qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

TypeError: __str__ returned non-string (type int)